In [2]:
!pip install opencv-python


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os 
import json
import time
import requests
import requests
from zipfile import ZipFile
import tarfile
from shutil import copyfile
from dataclasses import dataclass
import yaml
import glob
import random
import numpy as np
import pandas as pd
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/codespace/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
